In [32]:
import csv
import time
import numpy as np
import pandas as pd
import datetime as dt
from datetime import date
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import datetime
# import time

In [33]:
file = open("Feature_mapping.csv", "w", newline='', encoding="utf8")
f = csv.writer(file)

# Feature columns + label for each Userid
f.writerow(["Userid", "Still Morning", "Still Afternoon", "Still Evening", "Still Night","ADHD"])

69

In [34]:
df = pd.read_csv("Activity_clean.csv")
df = df.sort_values(by=['Userid','senseStartTimeMillis'],ascending=[True,True])

# get the list of ordered ids
ids = df["Userid"].unique()

In [35]:
# dictionary of ADHD labels for each user id
ADHD_dict = {}
baseline = pd.read_csv("sassema baseline 3.16.2023.csv")
baseline['ID'] = baseline['ID'].fillna(0)

for index in baseline.index:
    if index > 1:
        if baseline["ID"][index] != 0:
            if baseline["ID"][index] == "773":
                baseline["ID"][index] = "773684"
            if baseline["ADHDSx_Dx"][index] == "No":
                ADHD_dict[int(baseline["ID"][index])] = 0
            elif baseline["ADHDSx_Dx"][index] == "Yes":
                ADHD_dict[int(baseline["ID"][index])] = 1

In [36]:
def identifyTime(time,weekday):
    t = "23:59:59"
    night = datetime.datetime.strptime(t, "%H:%M:%S").time()
    t = "07:00:00"
    morning = datetime.datetime.strptime(t, "%H:%M:%S").time()
    t = "12:00:00"
    afternoon = datetime.datetime.strptime(t, "%H:%M:%S").time()
    t = "17:00:00"
    evening = datetime.datetime.strptime(t, "%H:%M:%S").time()
    time = time[1:-2]
    time_obj = datetime.datetime.strptime(time, "%H:%M:%S").time()

    if time_obj < morning:
        return "night"
    if time_obj >= morning and time_obj < afternoon:
        return "morning"
    if time_obj >= afternoon and time_obj < evening:
        return "afternoon"
    if time_obj >= evening and time_obj <= night:
        return "evening"

In [37]:
def determineWeekday(date):
    time = date
    time = time[2:-1]
    time = time.replace("T", " ")
    time = time.replace("-", "/")
    datetime_object = datetime.datetime.strptime(time, '%Y/%m/%d %H:%M:%S')
    dayOfTheWeek = datetime_object.weekday()

    if dayOfTheWeek >= 0 and dayOfTheWeek <= 4:
        weekday = 1
    else:
        weekday = 0
    
    return weekday, dayOfTheWeek
    

In [38]:
def CalculateStill(de_date, index, threshold, first, last, end_of_period, type_activity):
    thr = 0 
    toAdd = 0 

    # check if the element before is "Still"
    if first == 0:
        previousStill = 0
    else:
        if de_date['Activity'][index-1] != type_activity:
            previousStill = 0
        else:
            diff = de_date['senseStartTimeMillis'][index] - de_date['senseStartTimeMillis'][index-1]
            if diff > threshold:
                previousStill = 0
            else:
                previousStill = 1

    
    # check if the element after is "Still"
    if last == len(de_date['Activity']) - 1:
        afterStill = 0
    else:
        if de_date['Activity'][index+1] != type_activity:
            afterStill = 0
        else:
            diff = de_date['senseStartTimeMillis'][index+1] - de_date['senseStartTimeMillis'][index]
            if diff > threshold:
                afterStill = 0
            else:
                afterStill = 1
    
    # if the following element represent a different activity
    if afterStill == 0 and last != len(de_date['Activity']) - 1:
        diff = de_date['senseStartTimeMillis'][index+1] - de_date['senseStartTimeMillis'][index]
        if diff <= threshold:
            toAdd = diff
        else:
            toAdd = diff
            thr = 1
    
    # if the following element represent the same activity
    if afterStill == 1:
        diff = de_date['senseStartTimeMillis'][index+1] - de_date['senseStartTimeMillis'][index]
        toAdd = diff
     
    # if it is the last element  in the dataframe
    if last == len(de_date['Activity']) - 1:
        time = de_date['Final Date'][index] 
        time = time[1:-1]
        time = time.replace("-", "/")
        time += end_of_period
        convert = datetime.datetime.strptime(time,'%Y/%m/%d %H:%M:%S')
        
        epoch_time = convert.timestamp()
        gmt_epoch_time = int(epoch_time - datetime.timezone.utc.utcoffset(convert).total_seconds())*1000 - 4*3600000
        diff = gmt_epoch_time - de_date['senseStartTimeMillis'][index]

        if diff <= threshold:
            toAdd = diff
        else:
            toAdd = diff
            thr = 1
        
    return toAdd, thr, previousStill, afterStill


In [39]:
Userid = [] 
StillMorning  = []
StillAfternoon = []
StillEvening = []
StillNight = []
ADHD = []

In [42]:
threshold_night = 7*3600000
threshold_morning = 3600000
threshold_afternoon = 3600000
threshold_evening = 3*3600000
for id in ids:
    # get survey dates for the current id
    surveydates = df.loc[df["Userid"]==id, 'Survey Date'].tolist()
    uniquesurveydates = np.array(surveydates)
    uniquesurveydates = np.unique(uniquesurveydates)
    surveydates = uniquesurveydates.tolist()

    # get a dataframe of the data corresponding to current id
    de_id = df.loc[(df["Userid"])==int(id)]


    # calculate features for each survey date 
    for date in surveydates:
        # retrive dataframe that contains information for current survey date
        de_date = 0
        de_date = df.loc[(df["Userid"]==id) & (df["Survey Date"]==date)]
        # unique dates
        uniq = df.loc[(df["Userid"]==id) & (df["Survey Date"]==date), 'Final Date'].tolist()
        uniq = np.array(uniq)
        uniq = np.unique(uniq)
        uniq = uniq.tolist()
        
        # total time of being still for a survey date
        totalNightStill = 0
        totalMorningStill = 0
        totalAfternoonStill = 0
        totalEveningStill = 0
        toAddStillNight = 0
        toAddStillMorning = 0
        toAddStillAfternoon = 0
        toAddStillEvening = 0
        
        # keep track of first and last
        first = 0
        last = 0
        
        # iterate through dataframe and calculate features        
        for index in de_date.index:
            # determine weekend or weekday
            weekday, dayOfTheWeek = determineWeekday(de_date["senseStartTime"][index])
            
            # determine part of the day 
            partOfDay = identifyTime(de_date["Time"][index],weekday)
            
            if de_date["Activity"][index] == " 'Still'":
                # morning
                if (partOfDay == "morning"):
                    timeStill, thr, previousStill, afterStill = CalculateStill(de_date, index, threshold_morning, first, last, " 12:00:00", " 'Still'")
                    
                    # If it is a completely new bundle
                    if previousStill == 0:
                        # add bundle time to total time
                        totalMorningStill += toAddStillMorning
                        # reset bundle time
                        toAddStillMorning  = 0
                    # if thr has not been reached add to existing bundle 
                    if thr == 0:
                        toAddStillMorning += timeStill
                    # if thr has been reached 
                    
                    else:
                        if toAddStillMorning < threshold_morning:
                            if timeStill > threshold_morning:
                                toAddStillMorning = threshold_morning
                            else:
                                toAddStillMorning += timeStill
                        else:
                        # if the time in between activities > 3600 and total time to add is already > 3600 (add 10 more minutes)
                            toAddStillMorning += 600000
                    if last == len(de_date["Activity"]) - 1:
                        totalMorningStill += toAddStillMorning

                # afternoon
                elif (partOfDay == "afternoon"):
                    timeStill, thr, previousStill, afterStill = CalculateStill(de_date, index, threshold_afternoon, first, last, " 17:00:00", " 'Still'")
                    
                    # If it is a completely new bundle
                    if previousStill == 0:
                        # add bundle time to total time
                        totalAfternoonStill += toAddStillAfternoon
                        # reset bundle time
                        toAddStillAfternoon  = 0
                      
                    # if thr has not been reached add to existing bundle 
                    if thr == 0:
                        toAddStillAfternoon += timeStill
                    # if thr has been reached 
                    else:
                        if toAddStillAfternoon < threshold_afternoon:
                            if timeStill > threshold_afternoon:
                                toAddStillAfternoon = threshold_afternoon
                            else:
                                toAddStillAfternoon += timeStill
                        else:
                        # if the time in between activities > 3600 and total time to add is already > 3600 (add 10 more minutes)
                            toAddStillAfternoon += 600000
                                   
                    if last == len(de_date["Activity"]) - 1:
                        totalAfternoonStill += toAddStillAfternoon
                        
                # evening
                elif (partOfDay == "evening"):
                    timeStill, thr, previousStill, afterStill = CalculateStill(de_date, index, threshold_evening, first, last, " 23:59:59", " 'Still'")
                    
                    # If it is a completely new bundle
                    if previousStill == 0:
                        # add bundle time to total time
                        totalEveningStill += toAddStillEvening
                        # reset bundle time
                        toAddStillEvening  = 0
                      
                    # if thr has not been reached add to existing bundle 
                    if thr == 0:
                        toAddStillEvening += timeStill
                    # if thr has been reached 
                    else:
                        if toAddStillEvening < threshold_evening:
                            if timeStill > threshold_evening:
                                toAddStillEvening = threshold_evening
                            else:
                                toAddStillEvening += timeStill
                        else:
                        # if the time in between activities > 3600 and total time to add is already > 3600 (add 10 more minutes)
                            toAddStillEvening += 600000
                                   
                    if last == len(de_date["Activity"]) - 1:
                        totalEveningStill += toAddStillEvening
                
                # night
                elif (partOfDay == "night"):
                    timeStill, thr, previousStill, afterStill = CalculateStill(de_date, index, threshold_night, first, last, " 07:00:00", " 'Still'")
                    
                    # If it is a completely new bundle
                    if previousStill == 0:
                        # add bundle time to total time
                        totalNightStill += toAddStillNight
                        # reset bundle time
                        toAddStillNight  = 0
                      
                    # if thr has not been reached add to existing bundle 
                    if thr == 0:
                        toAddStillNight += timeStill
                    # if thr has been reached 
                    else:
                        if toAddStillNight < threshold_night:
                            if timeStill > threshold_night:
                                toAddStillNight = threshold_night
                            else:
                                toAddStillNight += timeStill
                        else:
                        # if the time in between activities > 3600 and total time to add is already > 3600 (add 10 more minutes)
                            toAddStillNight += 600000
                                   
                    if last == len(de_date["Activity"]) - 1:
                        totalNightStill += toAddStillNight
                    
            # keep track of first and last
            first = 1
            last += 1
        
        # calculate averages
        avgMorningStill = totalMorningStill / len(uniq)
        avgAfternoonStill = totalAfternoonStill / len(uniq)
        avgEveningStill = totalEveningStill / len(uniq)
        avgNightStill = totalNightStill / len(uniq)
        
        # append the averages to lists
        Userid.append(id) 
        StillMorning.append(avgMorningStill) 
        StillAfternoon.append(avgAfternoonStill) 
        StillEvening.append(avgEveningStill) 
        StillNight.append(avgNightStill)
        ADHD.append(ADHD_dict[id])
        
    print(id, " : ", StillMorning)
print(len(ADHD))
            


158455  :  [4114285.714285714, 1440000.0, 2057142.857142857, 2571428.5714285714, 4408281.333333333, 3011279.8333333335, 4114285.714285714, 4477135.428571428, 4487728.0, 814847.8, 3019262.1428571427, 3034965.0, 0.0, 0.0, 0.0, 1028571.4285714285, 4894351.571428572, 4845688.285714285, 4513522.285714285, 2171871.6666666665, 1631911.142857143, 5230254.0, 4186557.1666666665, 8604052.8, 5437196.2, 7492422.428571428, 5142857.142857143, 2349158.6666666665, 1800000.0, 3000000.0, 4011473.4285714286, 4628571.428571428, 5157181.571428572, 4396914.0, 4034063.1428571427, 4363344.75, 4676299.428571428, 6697973.142857143, 0.0, 5142857.142857143, 3600000.0, 3600000.0, 4847389.571428572, 5202396.571428572, 4335831.857142857, 6232508.428571428, 5194438.714285715, 6303911.0, 6748397.285714285, 3507012.5714285714, 4359061.285714285, 3640874.0, 2278630.8571428573, 4663057.142857143, 900000.0, 2633318.2, 5746663.6, 5216166.833333333, 6887872.857142857, 2879382.0, 6110215.285714285, 4186770.714285714, 5356916.